# Fisheries Monitoring

* [The Nature Conservancy Fisheries Monitoring | Kaggle](https://www.kaggle.com/c/the-nature-conservancy-fisheries-monitoring)

In [1]:
%matplotlib inline
from __future__ import print_function, division
#path = "data/state/"

import os, sys
sys.path.insert(1, os.path.join(sys.path[0], 'utils'))
import utils; reload(utils)
from utils import *
from IPython.display import FileLink

Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5103)
/home/ubuntu/anaconda2/lib/python2.7/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)
Using Theano backend.


##  Setup

In [4]:
%cd $path

[Errno 2] No such file or directory: '$path'
/home/ubuntu/nbs


In [3]:
%cd 'train'

[Errno 2] No such file or directory: 'train'
/home/ubuntu/nbs


In [19]:
import glob
import shutil
g = glob.glob('*/*.jpg')
shuf = np.random.permutation(g)
for i in range(777): 
    shutil.move(shuf[i], '../valid/' + shuf[i])

## Use Vgg16

In [5]:
batch_size = 64
path = "/home/ubuntu/datasets/the-nature-conservancy-fisheries-monitoring/"

In [6]:
vgg = Vgg16()

In [7]:
gen_t = image.ImageDataGenerator(rotation_range=15, height_shift_range=0.05, 
                shear_range=0.1, channel_shift_range=20, width_shift_range=0.1)

In [8]:
batches = vgg.get_batches(path+'train', gen_t, batch_size=batch_size)
val_batches = vgg.get_batches(path+'valid', batch_size=batch_size)

Found 3000 images belonging to 8 classes.
Found 777 images belonging to 8 classes.


In [11]:
vgg.finetune(batches)

In [12]:
vgg.fit(batches, val_batches, nb_epochs=1)

Epoch 1/1
3000/3000 [==============================] - 236s - loss: 2.3831 - acc: 0.4257 - val_loss: 1.2382 - val_acc: 0.5946


In [30]:
test_batches, preds = vgg.test(path+'test_stg1', batch_size=batch_size)
test_batches2, preds2 = vgg.test(path+'test_stg2', batch_size=batch_size)

Found 1000 images belonging to 1 classes.
Found 12153 images belonging to 1 classes.


## Submit

In [31]:
def do_clip(arr, mx): return np.clip(arr, (1-mx)/9, mx)

In [38]:
subm = do_clip(preds,0.93)
subm2 = do_clip(preds2, 0.93)

In [60]:
test_filename1 = [a[8:] for a in test_batches.filenames]
test_filename2 = [("test_stg2/"+a[8:]) for a in test_batches2.filenames]

In [62]:
classes = sorted(batches.class_indices, key=batches.class_indices.get)
preds = np.concatenate([subm, subm2])
submission = pd.DataFrame(subm, columns=classes)
test_filenames = np.concatenate([test_filename1, test_filename2])
submission.insert(0, 'image', test_filenames)
submission.head()

,image,ALB,BET,DOL,LAG,NoF,OTHER,SHARK,YFT
0,img_06237.jpg,0.712592,0.007778,0.007778,0.007778,0.024181,0.098804,0.007778,0.157347
1,img_06893.jpg,0.137833,0.007778,0.007778,0.007778,0.007778,0.007778,0.008331,0.850667
2,img_02082.jpg,0.161405,0.007778,0.007778,0.007778,0.385167,0.023748,0.220766,0.203721
3,img_06261.jpg,0.180560,0.007778,0.007778,0.007778,0.007778,0.007778,0.017289,0.799304
4,img_03628.jpg,0.806104,0.007778,0.007778,0.007778,0.008837,0.007778,0.007778,0.175503


In [63]:
submission.to_csv('submission.csv', index=False)

In [64]:
FileLink('submission.csv')

/home/ubuntu/nbs/submission.csv